In [1]:
!nvidia-smi

Fri Jun 11 12:20:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install git+https://github.com/finetuneanon/transformers@gpt-j

  Cloning https://github.com/finetuneanon/transformers (to revision gpt-j) to /tmp/pip-req-build-60a0rde1
  Running command git clone -q https://github.com/finetuneanon/transformers /tmp/pip-req-build-60a0rde1
  Running command git checkout -b gpt-j --track origin/gpt-j
  Switched to a new branch 'gpt-j'
  Branch 'gpt-j' set up to track remote branch 'gpt-j' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 4.3MB/s 
     |████████████████████████████████| 901kB 47.8MB/s 
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2405405 sha256=d1d1b0881340f630423fee6930deb7db1d6a21082657c1005a2c1163dc1054b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ljf2n8hr/wheels/04/ec/39/167ecac18e60370e8f800072b342efd79a13384f5f04172319
Successfully built transformers


In [3]:
import time
import torch
from transformers import GPTNeoForCausalLM
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").cuda()

In [2]:
print(model.dtype, model.device)
print("seq_len\tmax_len\truntime\tmemory")
for seq_len in [64, 127, 251, 256, 383, 509, 512, 769, 1021, 1024, 1280, 1283, 1536, 1543, 1792, 1801, 2039, 2048]:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  ids = torch.zeros(1, max(1, seq_len)).long().cuda()
  runtime = 0.
  max_length = min(2049, seq_len + 40)
  for i in range(10):
    s = time.perf_counter()
    outputs = model.generate(ids, use_cache=True, do_sample=True, min_length=max_length, max_length=max_length, pad_token_id=50256)
    runtime += time.perf_counter() - s
    del outputs
  print(f"{seq_len}\t{max_length}\t{runtime / 10.:.4f}s\t{torch.cuda.max_memory_allocated() / 1024. / 1024. / 1024.:.3f}GB")
  del ids

torch.float32 cuda:0
seq_len	max_len	runtime	memory
64	104	1.1515s	10.257GB
127	167	1.2066s	10.338GB
251	291	1.3745s	10.489GB
256	296	1.3792s	10.492GB
383	423	1.5356s	10.677GB
509	549	1.6660s	10.853GB
512	552	1.6667s	10.854GB
769	809	2.1005s	11.220GB
1021	1061	2.3348s	11.579GB
1024	1064	2.3344s	11.580GB
1280	1320	2.6338s	11.952GB
1283	1323	2.6533s	11.953GB
1536	1576	3.0467s	12.345GB
1543	1583	3.0944s	12.348GB
1792	1832	3.3902s	12.691GB
1801	1841	3.4388s	12.695GB
2039	2049	2.2853s	13.021GB
2048	2049	1.8421s	12.100GB


In [3]:
import transformers
transformers.__version__

'4.7.0.dev0'

In [4]:
# half precision run
model = model.half()
print(model.dtype, model.device)
print("seq_len\tmax_len\truntime\tmemory")
for seq_len in [64, 127, 251, 256, 383, 509, 512, 769, 1021, 1024, 1280, 1283, 1536, 1543, 1792, 1801, 2039, 2048]:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
  ids = torch.zeros(1, max(1, seq_len)).long().cuda()
  runtime = 0.
  max_length = min(2049, seq_len + 40)
  for i in range(10):
    s = time.perf_counter()
    outputs = model.generate(ids, use_cache=True, do_sample=True, min_length=max_length, max_length=max_length, pad_token_id=50256)
    runtime += time.perf_counter() - s
    del outputs
  print(f"{seq_len}\t{max_length}\t{runtime / 10.:.4f}s\t{torch.cuda.max_memory_allocated() / 1024. / 1024. / 1024.:.3f}GB")
  del ids

torch.float16 cuda:0
seq_len	max_len	runtime	memory
64	104	0.9781s	5.186GB
127	167	1.0396s	5.225GB
251	291	1.1590s	5.310GB
256	296	1.1628s	5.311GB
383	423	1.3393s	5.401GB
509	549	1.4969s	5.490GB
512	552	1.4975s	5.493GB
769	809	1.8752s	5.683GB
1021	1061	2.1957s	5.861GB
1024	1064	2.1975s	5.861GB
1280	1320	2.5928s	6.051GB
1283	1323	2.6369s	6.053GB
1536	1576	2.9966s	6.243GB
1543	1583	3.0621s	6.249GB
1792	1832	3.3844s	6.422GB
1801	1841	3.4636s	6.426GB
2039	2049	2.6127s	6.597GB
2048	2049	2.2140s	6.458GB
